In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window


silver_df = spark.table("silver.guests_validated")


dim_guests_df = 
    silver_df
         .filter(F.col("data_valid") == True)
         .select(
         "guest_name",
         "email",
         "guest_number",
         "phone_number",
         "street_address",
         "guest_date",
         "state"
      )
      .dropDuplicates()
      .withColumn(
        "guest_sk",
        F.row_number()
         .over(Window.orderBy("guest_number"))
      )
      .select("guest_sk", "guest_name",
         "email",
         "guest_number",
         "phone_number",
         "street_address",
         "guest_date",
         "state")


In [0]:
%sql SELECT current_catalog(), current_schema();

In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS gold")

In [0]:
dim_guests_df.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", True) \
    .option("path", "/mnt/gold/dim_guests/") \
    .saveAsTable("gold.dim_guests")




In [0]:
fact_state_df = (
        dim_guests_df.
        groupBy("state")
        .agg(F.count("guest_sk").alias("guest_count"))
        .withColumn("fact_state_sk",F.monotonically_increasing_id()+1 )
        .select("fact_state_sk", "state","guest_count")
)

fact_state_df.write.format("delta").mode("overwrite").option("overwriteSchema",True)\
    .option("path","/mnt/gold/fact_state/")\
    .saveAsTable("gold.fact_state")





In [0]:
%sql
SELECT guest_number, COUNT(DISTINCT guest_name) AS unique_guest_names
FROM gold.dim_guests
GROUP BY guest_number;

In [0]:
%sql
SELECT
    guest_number,
    COUNT(DISTINCT guest_name) AS unique_guest_names,
    ROUND(
        (COUNT(DISTINCT guest_name) * 100.0) / SUM(COUNT(DISTINCT guest_name)) OVER (),
        2
    ) AS percentage_of_total
FROM gold.dim_guests
GROUP BY guest_number;


In [0]:
%sql
WITH guest_counts AS (
    SELECT
        guest_number,
        COUNT(*) AS guest_number_count
    FROM gold.dim_guests
    GROUP BY guest_number
),
total_counts AS (
    SELECT
        SUM(guest_number_count) AS total_guest_numbers
    FROM guest_counts
)
SELECT
    g.guest_number,
    g.guest_number_count,
    ROUND((g.guest_number_count * 100.0) / t.total_guest_numbers, 2) AS percentage_of_total
FROM guest_counts g
CROSS JOIN total_counts t
ORDER BY percentage_of_total DESC;


In [0]:
%sql
SELECT
    guest_number,
    COUNT(*) AS guest_number_count,
    ROUND(
        COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (),
        2
    ) AS percentage_of_total
FROM gold.dim_guests
GROUP BY guest_number
ORDER BY percentage_of_total DESC LIMIT 10 ;


In [0]:
%sql
SELECT
    COUNT(DISTINCT guest_number) AS unique_guest_number_count,
    COUNT(*) AS total_record_count,
    ROUND(
        (COUNT(DISTINCT guest_number) * 100.0) / COUNT(*),
        2
    ) AS percentage_of_unique_guest_numbers
FROM gold.dim_guests;

In [0]:
%sql
SELECT
    COUNT(DISTINCT guest_name) AS unique_guest_names_count,
  
    ROUND(
        (COUNT(DISTINCT guest_name) * 100.0) / COUNT(*),
        2
    ) AS percentage_of_unique_guest_numbers
FROM gold.dim_guests;

In [0]:
%sql
SELECT
    COUNT(DISTINCT email) AS unique_email_count,
  
    ROUND((COUNT(DISTINCT email) * 100.0) / COUNT(*),2
    ) AS percentage_of_unique_guest_numbers
FROM gold.dim_guests;

In [0]:
%sql
SELECT
    COUNT(DISTINCT phone_number) AS unique_email_count,
  
    ROUND(
        (COUNT(DISTINCT phone_number) * 100.0) / COUNT(*),
        2
    ) AS percentage_of_unique_guest_numbers
FROM gold.dim_guests;

In [0]:
%sql
SELECT
    COUNT(DISTINCT street_address) AS unique_street_address_count,
  
    ROUND(
        (COUNT(DISTINCT street_address) * 100.0) / COUNT(*),
        2
    ) AS percentage_of_unique_street_address
FROM gold.dim_guests;

In [0]:
%sql
SELECT street_address, COUNT(*) AS occurrences
FROM gold.dim_guests
GROUP BY street_address
HAVING COUNT(*) > 1
ORDER BY occurrences DESC
LIMIT 20;

In [0]:
%sql
SELECT
   
    (SELECT COUNT(DISTINCT guest_number) FROM gold.dim_guests) AS distinct_guest_numbers,
    (SELECT COUNT(DISTINCT email) FROM gold.dim_guests) AS distinct_emails,
    (SELECT COUNT(DISTINCT phone_number) FROM gold.dim_guests) AS distinct_phone_numbers,

    -- Top 3 guest numbers 
    (
        SELECT array_join(array_agg(guest_number), ', ')
        FROM (
            SELECT guest_number
            FROM gold.dim_guests
            GROUP BY guest_number
            ORDER BY COUNT(*) DESC
            LIMIT 3
        )
    ) AS top_3_guest_numbers,

    -- Top 3 emails
    (
        SELECT array_join(array_agg(email), ', ')
        FROM (
            SELECT email
            FROM gold.dim_guests
            GROUP BY email
            ORDER BY COUNT(*) DESC
            LIMIT 3
        )
    ) AS top_3_emails,

     (
        SELECT array_join(array_agg(guest_name), ', ')
        FROM (
            SELECT guest_name
            FROM gold.dim_guests
            GROUP BY guest_name
            ORDER BY COUNT(*) DESC
            LIMIT 3
        )
    ) AS top_3_guest_names,

    -- Top 3 phone numbers
    (
        SELECT array_join(array_agg(phone_number), ', ')
        FROM (
            SELECT phone_number
            FROM gold.dim_guests
            GROUP BY phone_number
            ORDER BY COUNT(*) DESC
            LIMIT 3
        )
    ) AS top_3_phone_numbers